In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scienceplots
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import seaborn as sns

from IPython.display import display, clear_output
import warnings

from scipy.stats import skew, kurtosis

import os
import sys
from dotenv import load_dotenv

load_dotenv()

REPO_PATH = os.getenv('REPO_PATH')

sys.path.insert(0, rf'{REPO_PATH}src')

warnings.filterwarnings("ignore")
plt.style.use('science')

In [ ]:
FUTURES = ['CLc1', 'LCOc1']

dfs: dict[str, pd.DataFrame] = {
    future: pd.read_csv(
        rf'{REPO_PATH}data\raw_futures_data\{future}_High_Frequency.csv',
        index_col='Date',
        parse_dates=True
    ) for future in FUTURES
}

### Price plot

In [ ]:
LABELS = {
    'CLc1': 'NYMEX WTI Crude (CLc1)',
    'LCOc1': 'ICE Brent Crude (LCOc1)'
}

PALETTE = sns.color_palette('twilight', n_colors=2)

fig, ax = plt.subplots(figsize=(10,5), dpi=200)
for i, future in enumerate(FUTURES):
    dfs[future]['CLOSE'].plot(
        label=LABELS[future], 
        ax=ax, 
        lw=0.8, 
        color=PALETTE[i]
    )

ax.set_ylabel('Price (USD)', fontsize=14)
ax.set_xlabel('Date', fontsize=14)
ax.legend(frameon=False, loc='upper left', fontsize=13)
ax.tick_params(axis='x', labelsize=14)
ax.grid(alpha=0.2)

fig.savefig('images/crude_oil_price.png', bbox_inches='tight')

### Log returns analysis

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10,5), dpi=200, sharex=True)

df_list = []
for i, (key, df) in enumerate(dfs.items()):
    logret = np.log(df['CLOSE']).diff()
    logret_resampled = logret.resample('1D').sum() * 100
    df = pd.DataFrame(logret_resampled.describe()).T
    df['skew'] = skew(logret_resampled.dropna())
    df['kurtosis'] = kurtosis(logret_resampled.dropna())
    df.index = [LABELS[key]]
    df_list.append(df.T)

    axs[i].plot(logret * 100, lw=0.8, color=PALETTE[i], label=LABELS[key])
    axs[i].set_ylabel('Log Return (\%)', fontsize=16)
    axs[i].legend(frameon=False, loc='upper left', fontsize=15)

axs[-1].set_xlabel('Date', fontsize=16)
fig.tight_layout(pad=-0.5)

df = pd.concat(df_list, axis=1)

display(df)

fig.savefig('images/crude_oil_log_returns.png', bbox_inches='tight')

### Volume plot

In [ ]:
volume_df = pd.concat(
    [
        df['VOLUME'].resample('w').sum() 
        for df in dfs.values()
    ], axis=1
)

volume_df.columns = LABELS.values()

fig, ax = plt.subplots(figsize=(10,5), dpi=200)

volume_df.plot(
    kind='bar', 
    stacked=True, 
    ax=ax, 
    color=PALETTE, 
    width=0.8, 
    alpha=1
    )

ax.set_ylabel('Volume', fontsize=14)
ax.set_xlabel('Date', fontsize=14)
ax.legend(frameon=False, loc='upper left', fontsize=13)
ax.tick_params(axis='x', labelsize=14)

plt.xticks(
    np.arange(0, len(volume_df), 4), 
    volume_df.index[np.arange(0, len(volume_df), 4)].strftime('%Y-%m'), 
    rotation=45
)

fig.savefig('images/crude_oil_volume_stacked.png', bbox_inches='tight')

### Price last 20 years

In [ ]:
df_CLc1_daily = pd.read_csv(
    rf'{REPO_PATH}data\raw_futures_data\CLc1.csv',
    index_col='Date',
    parse_dates=True
)
df_LCOc1_daily = pd.read_csv(
    rf'{REPO_PATH}data\raw_futures_data\LCOc1.csv',
    index_col='Date',
    parse_dates=True
)

fig, ax = plt.subplots(figsize=(10,4), dpi=200)

df_CLc1_daily['CLOSE'].plot(label=LABELS['CLc1'], ax=ax, lw=0.6, color=PALETTE[0])
df_LCOc1_daily['CLOSE'].plot(label=LABELS['LCOc1'], ax=ax, lw=0.6, color=PALETTE[1])
ax.set_ylabel('Price (USD)', fontsize=14)
ax.set_xlabel('Date', fontsize=14)
ax.legend(frameon=False, loc='lower left', fontsize=13)
ax.tick_params(axis='x', labelsize=12)
ax.grid(alpha=0.2)
ax.set_axisbelow(True)

fig.savefig('images/crude_oil_price_daily.png', bbox_inches='tight')
